In [2]:
import os
import ssl
import time
import socket
import requests
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from urllib import parse
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup
#from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager #!pip install webdriver-manager

# 한글대응
from matplotlib import rc
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
rc('font',family='Malgun Gothic')

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def create_folder(directory):
    import os
    
    if not os.path.exists(directory):
        os.mkdir(directory)

In [4]:
category_dict = {
    '치킨' : 5,
    '피자-양식' : 6,
    '중국집' : 7,
    '한식' : 8,
    '일식-돈까스' : 9,
    '족발-보쌈' : 10,
    '야식' : 11,
    '분식' : 12,
    '카페-디저트' : 13,
}

# 샐러드 리뷰 이미지 크롤링 (그리바스켓)

In [5]:
def salad_crawling(get_review_num, save_dir, menu):
    # 사이트 접속
    url = 'https://www.yogiyo.co.kr/mobile/#/'
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url); action = ActionChains(driver); time.sleep(1)
    driver.maximize_window(); time.sleep(1)

    #검색창 선택 -> 주소입력
    CSS_SELECTOR = '#search > div > form > input'
    search = driver.find_element(By.CSS_SELECTOR, CSS_SELECTOR)
    search.click()
    search.send_keys(Keys.CONTROL + "A")
    search.send_keys("서초대로38길 12, 마제스타시티")

    # 검색버튼 클릭
    button_click = driver.find_element(By.CSS_SELECTOR, "#button_search_address > button.btn.btn-default.ico-pick")
    button_click.click()
    time.sleep(1)

    #첫번째 주소 클릭
    address_click = driver.find_element(By.CSS_SELECTOR, "#search > div > form > ul > li:nth-child(3) > a")
    address_click.click()
    time.sleep(1)

    #돋보기 클릭
    button_click2 = driver.find_element(By.CSS_SELECTOR, "#category > ul > li.hidden-xs.menu-search > a")
    button_click2.click() ; time.sleep(1)

    #샐러드 입력
    search2 = driver.find_element(By.CSS_SELECTOR, "#category > ul > li.main-search > form > div > input")
    search2.click()
    search2.send_keys("만다린")
    time.sleep(1)

    #검색버튼 클릭
    driver.find_element(By.CSS_SELECTOR, "#category_search_button").click(); time.sleep(1)


    #그린바스캣 클릭
    driver.find_element(By.CSS_SELECTOR, "#content > div > div:nth-child(5) > div > div > div > div").click()

    # 리뷰탭 클릭
    review_btn = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(2) > a').click()
    time.sleep(1)
    
    # 사진 리뷰만 보기
    review_img = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div:nth-child(6) > div.review-count > div > div > div > label').click()
    time.sleep(1)
    
    # 스크롤 맨 아래로
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(1)  # 브라우저 끝까지 스크롤을 내림
    
    review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
    
    while (review_num < get_review_num):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 브라우저 끝까지 스크롤을 내림
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, '#review > li.list-group-item.btn-more > a').click()
        time.sleep(1)
        review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
        
    # 현재 화면의 html 코드를 가져와서 뷰숲으로 작업하기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    
    # 리뷰 목록 불러오기
    review_tables = soup.select('#review>li')[1:get_review_num+1]
    
    make_save_folder = save_dir + 'reviews_data'
    create_folder(make_save_folder)

    for n, review in enumerate(review_tables):
        img_tag = review.select('table>tbody>tr>td>div>img')
        image_urls = [each['data-url'] for each in img_tag]

        make_user_folder = make_save_folder+'/'+menu+'reviewer'+str(n+1)
        create_folder(make_user_folder)

        for idx, url in enumerate(image_urls):
            response = requests.get(url)

            if response.status_code == 200:
                image_file = os.path.join(make_user_folder, f'img{idx+1}.jpg')
                with open(image_file, 'wb') as file:
                    file.write(response.content)
                #print('이미지 저장', image_file)
            else:
                print('이미지 저장 실패')
                
    driver.quit()

In [6]:
salad_crawling(get_review_num=100, save_dir='../Data_total/new_JjamBbong/', menu='짬뽕')

# 가게이름 크롤링

In [67]:
def yogiyo_crawling(store,get_review_num, save_dir, menu):
    url = 'https://www.yogiyo.co.kr/mobile/#/'
    driver = webdriver.Chrome('../chromedriver.exe')
    driver.get(url); action = ActionChains(driver); time.sleep(1)

    driver.maximize_window(); time.sleep(1)

    #검색창 선택 -> 주소입력
    CSS_SELECTOR = '#search > div > form > input'
    search = driver.find_element(By.CSS_SELECTOR, CSS_SELECTOR)
    search.click()
    search.send_keys(Keys.CONTROL + "A")
    search.send_keys("서초대로38길 12, 마제스타시티")

    # 검색버튼 클릭
    button_click = driver.find_element(By.CSS_SELECTOR, "#button_search_address > button.btn.btn-default.ico-pick")
    button_click.click()
    time.sleep(1)

    #첫번째 주소 클릭
    address_click = driver.find_element(By.CSS_SELECTOR, "#search > div > form > ul > li:nth-child(3) > a")
    address_click.click()
    time.sleep(1)
    
    #돋보기 클릭
    button_click2 = driver.find_element(By.CSS_SELECTOR, "#category > ul > li.hidden-xs.menu-search > a")
    button_click2.click() ; time.sleep(1)

    #샐러드 입력
    search2 = driver.find_element(By.CSS_SELECTOR, "#category > ul > li.main-search > form > div > input")
    search2.click()
    search2.send_keys(store)
    time.sleep(1)

    #검색버튼 클릭
    driver.find_element(By.CSS_SELECTOR, "#category_search_button").click(); time.sleep(1)

    #죽이요 클릭
    driver.find_element(By.CSS_SELECTOR, "#content > div > div:nth-child(5) > div > div > div > div").click()
    
    # 리뷰탭 클릭
    review_btn = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(2) > a').click()
    time.sleep(1)
    
    # 사진 리뷰만 보기
    review_img = driver.find_element(By.CSS_SELECTOR, '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div:nth-child(6) > div.review-count > div > div > div > label').click()
    time.sleep(1)
    
    # 스크롤 맨 아래로
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(1)  # 브라우저 끝까지 스크롤을 내림
    
    review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
    
    while (review_num < get_review_num):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 브라우저 끝까지 스크롤을 내림
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, '#review > li.list-group-item.btn-more > a').click()
        time.sleep(1)
        review_num = len(driver.find_elements(By.XPATH, '//*[@id="review"]/li'))
        
    # 현재 화면의 html 코드를 가져와서 뷰숲으로 작업하기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    
    # 리뷰 목록 불러오기
    review_tables = soup.select('#review>li')[1:get_review_num+1]
    
    make_save_folder = save_dir + 'reviews_data'
    create_folder(make_save_folder)

    for n, review in enumerate(review_tables):
        img_tag = review.select('table>tbody>tr>td>div>img')
        image_urls = [each['data-url'] for each in img_tag]

        make_user_folder = make_save_folder+'/'+menu+'reviewer'+str(n+1)
        create_folder(make_user_folder)

        for idx, url in enumerate(image_urls):
            response = requests.get(url)

            if response.status_code == 200:
                image_file = os.path.join(make_user_folder, f'img{idx+1}.jpg')
                with open(image_file, 'wb') as file:
                    file.write(response.content)
                print('이미지 저장', image_file)
            else:
                print('이미지 저장 실패')
                
    driver.quit()

In [62]:
yogiyo_crawling(store = "죽이요",get_review_num=100, save_dir='../Data/', menu='죽')

이미지 저장 ../Data/reviews_data/죽reviewer1\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer1\img2.jpg
이미지 저장 ../Data/reviews_data/죽reviewer2\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer3\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer4\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer5\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer6\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer7\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer8\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer9\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer10\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer11\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer12\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer13\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer13\img2.jpg
이미지 저장 ../Data/reviews_data/죽reviewer13\img3.jpg
이미지 저장 ../Data/reviews_data/죽reviewer14\img1.jpg
이미지 저장 ../Data/reviews_data/죽reviewer14\img2.jpg
이미지 저장 ../Data/reviews_data/죽reviewer14\img3.jpg
이미지 저장 ../Data/reviews_data/죽reviewer15\img1.jpg
이미지 저장 ../Data/reviews_data/죽r

# 족발 크롤링

In [66]:
yogiyo_crawling(store ="기대만족" ,get_review_num=100, save_dir='../Data/', menu='족발')

이미지 저장 ../Data/reviews_data/족발reviewer1\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer2\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer2\img2.jpg
이미지 저장 ../Data/reviews_data/족발reviewer2\img3.jpg
이미지 저장 ../Data/reviews_data/족발reviewer3\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer4\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer5\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer6\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer7\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer8\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer9\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer9\img2.jpg
이미지 저장 ../Data/reviews_data/족발reviewer10\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer11\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer12\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer13\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer14\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer14\img2.jpg
이미지 저장 ../Data/reviews_data/족발reviewer15\img1.jpg
이미지 저장 ../Data/reviews_data/족발reviewer16\img1.jpg
이미지 저장 ../Da